In [ ]:
from pathlib import Path
result = list(Path("flaskblog").rglob("*.json"))
for i in range(1,len(result)):
    x = str(result[i])
    print(x)
    #print(x.replace('\\','/'))

In [ ]:
print(len(result))

In [ ]:
import json
static = {}
count_covid = 0
count_non_covid = 0
count_india = 0
count_italy = 0
count_usa = 0
poi_names = {}
for i in range(1,len(result)):
    #print(result[i])
    try:
        x = str(result[i])
        x = x.replace('\\','/')
        with open(x,encoding='utf-8') as f:
            data = f.read()
        if 'JSON_IR_ROHIT' not in x:
            data = data.replace('}{','},{')
            data = '['+data+']'
            posts = json.loads(data)
        
        else:
            posts = json.loads(data)
        
        for post in posts:
            if ('covid' or 'corona') in post['full_text'].lower():
                count_covid+=1
            else:
                count_non_covid+=1
                
            if post['country'].lower()=='india':
                count_india+=1
            elif post['country'].lower()=='italy':
                count_italy+=1
            else:
                count_usa+=1
            if(post['user']['screen_name']!=None):
                if post['user']['screen_name'].lower() not in poi_names:
                    poi_names[post['user']['screen_name'].lower()]=1
                else:
                    poi_names[post['user']['screen_name'].lower()]+=1
        f.close()
    except:
        print(x)
        pass

In [ ]:
print(count_covid)
print(count_non_covid)
print(poi_names)
print(count_india)
print(count_usa)
print(count_italy)

In [ ]:
poi = {}
for key,value in poi_names.items():
    if(poi_names[key]>500):
        poi[key] = value
print(poi)
data = {}
data["count_covid"] = count_covid
data["count_non_covid"] = count_non_covid
data["poi_names"] = poi
data["count_india"] = count_india
data["count_usa"] = count_usa
data["count_italy"] = count_italy
with open('static.json','w') as f:
    json.dump(data,f)

In [ ]:
import pandas as pd

for i in range(1,len(result)):
    try:
        x = str(result[i])
        x = x.replace('\\','/')
        with open(x,encoding='utf-8') as f:
            data = f.read()
        if 'JSON_IR_ROHIT' not in x:
            data = data.replace('}{','},{')
            data = '['+data+']'
            posts = json.loads(data)
        
        else:
            posts = json.loads(data)
    except:
        print(result[i])


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

from google_trans_new import google_translator

def sentiment_score_function(text):
    #gs = google_translator()
    #text = gs.translate(text,'en')
    return analyser.polarity_scores(text)['compound']

In [ ]:
import pandas as pd
import json
from datetime import datetime


json_dataframe = pd.DataFrame()

simple_list = []
created_at = []
full_text = []
tweet_id = []
user = []
entities = []
verified = []
retweet_count = []
favorite_count = []
influencer_score = []
sentiment_score = []
country = []

for i in range(1,len(result)):

    try:
        x = str(result[i])
        x = x.replace('\\','/')
        with open(x,encoding='utf-8') as f:
            data = f.read()
        if 'JSON_IR_ROHIT' not in x:
            data = data.replace('}{','},{')
            data = '['+data+']'
            posts = json.loads(data)
        
        elif 'static' not in x:
            posts = json.loads(data)
        
        for post in posts:
            post = json.dumps(post)
            post = json.loads(post)
            created_at.append(post['created_at'])
            full_text.append(post['full_text'])
            tweet_id.append(post['id'])
            entities.append(post['entities'])
            verified.append(post['verified'])
            retweet_count.append(post['retweet_count'])
            favorite_count.append(post['favorite_count'])
            country.append(post['country'])
            influencer_score.append(post['favorite_count']+post['retweet_count'])
            sentiment_score.append(sentiment_score_function(post['full_text']))
            
        
    except Exception as e:
        print(e)
        print(result[i])

In [ ]:
df.to_csv('final.csv',header=True,index=False)

In [ ]:
print(x)

In [ ]:
def influencer_score(v1,v2,v3):
    if(v3):
        return (v1+v2)**2
    else:
        return v1+v2

In [ ]:
import pandas as pd
import json
from datetime import datetime

df = pd.DataFrame()

for i in range(0,len(result)):
    try:
        x = str(result[i])
        x = x.replace('\\','/')
        with open(x,encoding='utf-8') as f:
            data = f.read()
            
        if 'JSON_IR_ROHIT' not in x:
            data = data.replace('}{','},{')
            data = '['+data+']'
            posts = json.loads(data)
        
        elif 'static' not in x:
            posts = json.loads(data)
            
        
        df2 = pd.DataFrame(posts)
        df2 = df2[['created_at','full_text','id','user','entities','retweet_count','favorite_count','country']]
        if(df2.get("user").get("verified")):
            df2['influencer_score'] = (df2['favorite_count']+df2['retweet_count'])**2
        else:
            df2['influencer_score'] = df2['favorite_count']+df2['retweet_count']
    #df2['nScore'] = df2['influencer_score']/df2['influencer_score'].max()
        if(df.shape==(0,0)):
            df = df2
        else:
            df = pd.concat([df,df2])

        df.drop_duplicates(subset=['id'])
        
    except Exception as e:
        print(e)
        print(result[i])

In [ ]:
out = df.to_json(orient='records')[1:-1]
out = '[' + out + ']'

with open('outfile.json','w') as f:
    json.dump(json.loads(out),f)

In [ ]:
f.close()

In [ ]:
df.to_csv('final.csv',index=False)

In [ ]:
import pandas as pd
data = pd.read_csv('final.csv')
#data.to_json('final_json.json',orient='records')

In [ ]:
x = {}
import ast
import json
for i in range(0,len(data)):
    y = json.loads(json.dumps(ast.literal_eval(data['user'][i])))
    
    if ('corona' or 'coronavirus' or 'covid' or 'policy') in data['full_text'][i].lower():
        
        if(y['screen_name']!=None):
            if(y['verified']):
                if y['screen_name']+str(data['created_at'][i].split('T')[0]) not in x:
                    x[y['screen_name']+str(data['created_at'][i].split('T')[0])]=1
                else:
                    x[y['screen_name']+str(data['created_at'][i].split('T')[0])]+=1
    else:
        if(y['verified']):
            if(y['screen_name']!=None):
                if y['screen_name']+'-non'+str(data['created_at'][i].split('T')[0]) not in x:
                    x[y['screen_name']+'-non'+str(data['created_at'][i].split('T')[0])]=1
                else:
                    x[y['screen_name']+'-non'+str(data['created_at'][i].split('T')[0])]+= 1

In [ ]:
for key,value in sorted(x.items()):
    if key!=None:
        if value>10:
            print(key+' '+str(x[key]))